# Models Exploration using CUB dataset

## References
* [Transfer Learning with Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub)
* [`tf.keras.utils.image_dataset_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)
* [Limiting GPU Memory Growth](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)

## Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import datetime
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub
from keras.utils.layer_utils import count_params


In [2]:
def limit_memory_growth(limit=True):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, limit)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

In [3]:
limit_memory_growth()

1 Physical GPUs, 1 Logical GPUs


## Utility

In [4]:
def plot_predictions(
    image_batch,
    predicted_class_names,
):
    plt.figure(figsize=(10,9))
    plt.subplots_adjust(hspace=0.5)
    for n in range(30):
        plt.subplot(6,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(predicted_class_names[n])
        plt.axis('off')
    _ = plt.suptitle("Predictions")

def plot_images(
    ds,
    class_names,
):
    plt.figure(figsize=(10, 10))
    for images, labels in ds.tjake(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")
    
def get_timestamp():
    return datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

## Enumerate Datasets to test

In [5]:
import pathlib

flowers_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
flowers_data_dir = tf.keras.utils.get_file('flower_photos', origin=flowers_dataset_url, untar=True)
flowers_data_dir = pathlib.Path(flowers_data_dir)

datasets = [
    '/mnt/cub/CUB_200_2011/images',
    flowers_data_dir,
]

## Dataset

In [6]:
# build a dataset object from a directory of images
def build_dataset(
    data_dir,
    image_size,
    preprocess_input = None,
    batch_size = 64,
):
   
    train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        batch_size = batch_size,
        validation_split = 0.2,
        image_size = image_size,
        subset = "both",
        shuffle = True, # default but here for clarity
        seed=42,
        label_mode="categorical" # enables one-hot encoding (use 'int' for sparse_categorical_crossentropy loss)
    )
    
    # Retrieve class names
    # (can't do this after converting to PrefetchDataset?)
    class_names = train_ds.class_names
    
    # Prefetch images
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    # apply preprocessing function
    train_ds.map(
        lambda x, y: (preprocess_input(x), y),
    )
    
    return (train_ds, val_ds, class_names)

## Enumerate Models to test

In [7]:
# Array of tuples describing the models to be tested
# in the form: (model_handle, input_image_size, preprocessing_function)
# where the model_handle is a model building function or a url to a tfhub feature model
base_models_metadata = [
    # ('https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4', 224),
    # ('https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4', 299),
    # ('https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5', 299),
    (tf.keras.applications.Xception, 299, tf.keras.applications.xception.preprocess_input),
    # (tf.keras.applications.resnet.ResNet101, 224),
    # (tf.keras.applications.ResNet50, 224),
    # (tf.keras.applications.InceptionResNetV2, 299),
    # (tf.keras.applications.efficientnet_v2.EfficientNetV2B0, 224)
]

# return a name that accurately describes the model building function or
# the tfhub model (by url) that was passed
def get_model_name( model_handle ):
    
    if callable(model_handle):
        return f'keras.applications.{model_handle.__name__}'
    else:
        split = model_handle.split('/')
        return f'{split[-5]}.{split[-4]}.{split[-3]}'
    

## Model Building

In [8]:
# Print model weight counts
def print_weight_counts(model):
    print(f'Full Model - Non-trainable weights: {count_params(model.non_trainable_weights)}')
    print(f'Full Model - Trainable weights: {count_params(model.trainable_weights)}')
    
def build_base_model_layer(
    model_handle,
    name="base_model_layer",
):
    # If model_handle is a model building function, use that function
    if callable(model_handle):
        base_model_layer = model_handle(
            include_top=False,
            weights='imagenet',
            pooling = 'avg',
        )
        # Freeze base model
        base_model_layer.trainable = False
    # otherwise build a layer from the tfhub url that was passed as a string
    else:
        base_model_layer = hub.KerasLayer(
            model_handle,
            name=name,
            trainable = False, # default but here for clarity
        )
    
    # Print Base model weights
    print("Base Model:")
    print_weight_counts(base_model_layer)
    print()
    
    return base_model_layer

def build_model(
    base_model_metadata,
    num_classes,
    dropout,
):
    # Get base_model_information
    model_handle, input_dimension, preprocess_input = base_model_metadata

    # Build Full model
    model = Sequential([
        
        build_base_model_layer(
            model_handle,
        ),
        
        layers.Dense(
            num_classes,
            # activation = 'softmax',
        ),
        
        layers.Dropout(dropout),
        layers.Activation("softmax", dtype="float32"),

    ])
    
    # Print weight counts
    print("Full Model:")
    print_weight_counts(model)
    print()
    
    return model

## Build and run all models

In [9]:
# Directory for logs
base_log_dir = f'models_cub_02_logs'

In [10]:
# Hyperparameters
batch_size = 64
max_epochs = 5
dropout = 0.4
learning_rate = 0.0001


# for each base model
for base_model_metadata in base_models_metadata:
    
    model_handle, input_dimension, preprocess_input = base_model_metadata

    image_size = (input_dimension, input_dimension)
    
    # Build dataset/pipeline
    train_ds, val_ds, class_names = build_dataset(
        datasets[0],
        batch_size = batch_size,
        image_size = image_size,
        preprocess_input = preprocess_input,
    )
    
    # Build model
    model = build_model(
        base_model_metadata,
        len(class_names),
        dropout,
    )
    
    # Compile model
    # Sparse vs non-sparse CCE https://www.kaggle.com/general/197993
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
        # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        loss=tf.keras.losses.CategoricalCrossentropy(
            # from_logits=True,
        ),
        metrics=[
            'accuracy',
            # tf.keras.metrics.SparseCategoricalAccuracy(),
            # tf.keras.metrics.SparseCategoricalCrossentropy(from_logits=True),
            # tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="Top3"),
            # tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name="Top10"),
        ],
    )
            
    # Logging
    model_id = get_model_name(model_handle)
    log_dir = os.path.join( base_log_dir, model_id )
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
    )
    
    # Early stopping
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        # monitor='val_sparse_categorical_accuracy',
        monitor='accuracy',
        patience=5,
        min_delta=0.001,
    ),
    
    # Train
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=max_epochs,
        callbacks=[
            tensorboard_callback,
            early_stopping_callback,
        ]
    )
    
    # 
    
    # Save model
    # model.save(os.path.join(log_dir, 'final_model' ))    
    

Found 11788 files belonging to 200 classes.
Using 9431 files for training.
Using 2357 files for validation.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Base Model:
Full Model - Non-trainable weights: 20861480
Full Model - Trainable weights: 0

Full Model:
Full Model - Non-trainable weights: 20861480
Full Model - Trainable weights: 409800

Epoch 1/5
148/148 [==============================] - 33s 195ms/step - loss: 39.6085 - accuracy: 0.0061 - val_loss: 16.5337 - val_accuracy: 0.0072
Epoch 2/5
148/148 [==============================] - 24s 160ms/step - loss: 17.9117 - accuracy: 0.0066 - val_loss: 13.9473 - val_accuracy: 0.0064
Epoch 3/5
148/148 [==============================] - 24s 162ms/step - loss: 14.9294 - accuracy: 0.0104 - val_loss: 12.6120 - val_accuracy: 0.0068
Epoch 4/5
148/148 [==============================] - 24s 160ms/s